Now that we understand the indexing pipeline, we can
utilize our vector database to retrieve relevant documents
for a given query and use them to generate a response.

In [1]:
%pip install -qU langchain langchain_community langchain_huggingface langchain_openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# load the environment variables
%pip install -qU python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)

c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Users\kirth\OneDrive\Desktop\AI_Software_Engineer_Kirthika\Github_Projects_for_AI\RAG-LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


Embeddings - OpenAI Embeddings model
Vector store - In Memory VS

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
store=InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = store.as_retriever()

In [6]:
# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the US Constitution. Use the provided context to answer the question. IMPORTANT: If you are unsure of the answer, say 'I don't know' and don't make up an answer."),
    ("user", "Question: {question}\nContext: {context}")
])

chain = prompt | llm

## RAG

In [8]:
query = "What does the constitution say about pardons?"
# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs_content})

print(response.content)

The U.S. Constitution provides the President with the power to grant reprieves and pardons for offenses against the United States, except in cases of impeachment. This power is found in Article II, Section 2, Clause 1 of the Constitution.


In [9]:
#checking if I don't know works
query = "Is there any plans to change the US Constitution to allow for term limits on senators?"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs_content})

print(response.content)

I don't know. The context provided does not include any information about plans to change the US Constitution to allow for term limits on senators. Changes to the Constitution would require a formal proposal and ratification process, which is not detailed here.


## Using a better prompt:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:

src: https://smith.langchain.com/hub/rlm/rag-prompt

In [10]:
# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langsmith import Client

llm = ChatOpenAI(model_name="gpt-4o")

client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

chain = prompt | llm

In [11]:
query = "What does the constitution say about pardons?"
# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs_content})

print(response.content)

The Constitution states that the President may grant reprieves and pardons for offenses, except in cases of impeachment.


In [12]:
#checking if I don't know works
query = "Is there any plans to change the US Constitution to allow for term limits on senators?"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs_content})

print(response.content)

The provided context does not indicate any plans to change the US Constitution to allow for term limits on senators.


Multiple query RAG:

In [13]:
while True:
    query = input("Enter a question: ")
    if query == "exit":
        break
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})
    print(response.content)

The First Amendment to the United States Constitution prohibits Congress from making laws that establish a religion or restrict the free exercise of religion, freedom of speech, freedom of the press, the right of people to peaceably assemble, and the right to petition the Government for a redress of grievances. It was part of the first ten amendments proposed by the First Congress on September 25, 1789, and was ratified by several states shortly thereafter.
The Second Amendment to the United States Constitution ensures that a "well regulated Militia, being necessary to the security of a free State, the right of the people to keep and bear Arms, shall not be infringed."
